# Linear Web Crawls with Dates

In this section, we will look at using the `datetime` modules to create many urls that are based on dates.

## The Problem

* We have to create many urls based on dates/time.
    * Example: `http://www.thecurrent.org/playlist/2014-01-01/01`
* Working with dates is tough
    * different days per month
    * leap years

## The solution

We can use the `datetime` module to

* Create many dates
* Print to a specific output

In [1]:
import datetime

## Example - Scraping The Current

In a previous lab, we wrote a function get the information for a single page (1 hour of songs).  Here is the resulting functions.

In [37]:
import requests
from bs4 import BeautifulSoup

def get_period(soup):
    search = soup('span', class_="hour-header open")
    if len(search) > 0:
        return search[0].next.split('to')[0].rstrip()[-2:]
    else:
        return None

def get_day(soup):
    search = soup('a', class_="start-picker")
    if len(search) > 0:
        return search[0].next.split(',')[0]
    else:
        return None
    
def get_song_info(url):
    print("Starting {0} urls".format(url))
    date = url.split('/')[-2]
    s = requests.Session()
    r = s.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    period = get_period(soup)
    day_of_week = get_day(soup)
    soup = BeautifulSoup(r.content)
    titles = [t.next.strip() for t in soup.findAll('h5', class_="title")]
    artists = [a.next.strip() for a in soup.findAll('h5',class_='artist')]
    times = [d.time.next.strip() for d in soup('div', class_="two columns songTime")]
    song_info = [(day_of_week, date, time, period, title, artist) 
             for time, title, artist in zip(times, titles, artists)]
    return song_info

# Collecting a weeks worth of music

## Step 1 - Identify the url pattern

The Current uses urls of the following pattern

    'http://www.thecurrent.org/playlist/2017-05-04/10'

or 

    'http://www.thecurrent.org/playlist/year-month-day/hour'

## Question: How might you generate all combinations for a given year?

**Answer:** Python has a tool for that!

In [32]:
numdays = 7
# Set the base url as today
base = datetime.datetime.today()
# Make new times by subracting an hour at a time.
dts = [base - datetime.timedelta(hours = h) for h in range(0, numdays*24)]
dts

[datetime.datetime(2018, 10, 7, 11, 54, 10, 214064),
 datetime.datetime(2018, 10, 7, 10, 54, 10, 214064),
 datetime.datetime(2018, 10, 7, 9, 54, 10, 214064),
 datetime.datetime(2018, 10, 7, 8, 54, 10, 214064),
 datetime.datetime(2018, 10, 7, 7, 54, 10, 214064),
 datetime.datetime(2018, 10, 7, 6, 54, 10, 214064),
 datetime.datetime(2018, 10, 7, 5, 54, 10, 214064),
 datetime.datetime(2018, 10, 7, 4, 54, 10, 214064),
 datetime.datetime(2018, 10, 7, 3, 54, 10, 214064),
 datetime.datetime(2018, 10, 7, 2, 54, 10, 214064),
 datetime.datetime(2018, 10, 7, 1, 54, 10, 214064),
 datetime.datetime(2018, 10, 7, 0, 54, 10, 214064),
 datetime.datetime(2018, 10, 6, 23, 54, 10, 214064),
 datetime.datetime(2018, 10, 6, 22, 54, 10, 214064),
 datetime.datetime(2018, 10, 6, 21, 54, 10, 214064),
 datetime.datetime(2018, 10, 6, 20, 54, 10, 214064),
 datetime.datetime(2018, 10, 6, 19, 54, 10, 214064),
 datetime.datetime(2018, 10, 6, 18, 54, 10, 214064),
 datetime.datetime(2018, 10, 6, 17, 54, 10, 214064),
 da

## Outputing the time in a specific format.

* datetime objects have a `strftime` method for outputing formatted dates
    * `%Y` is the year in 4 digits
    * `%m` is the month in 2 digits
    * etc.
* See [this site](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior) for more information.

In [33]:
fmt = 'http://www.thecurrent.org/playlist/%Y-%m-%d/%H'

date = datetime.datetime(2000,1,1,1)
date.strftime(fmt)

'http://www.thecurrent.org/playlist/2000-01-01/01'

## Let's make some functions

In [34]:
def output_address(dt):
    fmt = 'http://www.thecurrent.org/playlist/%Y-%m-%d/%H'
    return dt.strftime(fmt)

def test_output_address():
    date = datetime.datetime(2000,1,1,1)
    assert output_address(date) == 'http://www.thecurrent.org/playlist/2000-01-01/01'
test_output_address()

In [35]:
urls = [output_address(d) for d in dts]
urls[:10]

['http://www.thecurrent.org/playlist/2018-10-07/11',
 'http://www.thecurrent.org/playlist/2018-10-07/10',
 'http://www.thecurrent.org/playlist/2018-10-07/09',
 'http://www.thecurrent.org/playlist/2018-10-07/08',
 'http://www.thecurrent.org/playlist/2018-10-07/07',
 'http://www.thecurrent.org/playlist/2018-10-07/06',
 'http://www.thecurrent.org/playlist/2018-10-07/05',
 'http://www.thecurrent.org/playlist/2018-10-07/04',
 'http://www.thecurrent.org/playlist/2018-10-07/03',
 'http://www.thecurrent.org/playlist/2018-10-07/02']

In [38]:

info_lists = [get_song_info(url) for url in urls]
flat_info = [row for info_list in info_lists for row in info_list]
lines = [",".join(row) for row in flat_info]

with open('the_current_last_year.csv', 'w') as outfile:
    header = "Weekday,Date,Time,Period,Song_Title,Artist"
    print(header, file=outfile)
    count = 0
    for url in urls:
        for line in lines:
            count += 1
            if count % 5 == 0:
                print("Processed {0} songs".format(count))
            print(line, file=outfile)

Starting http://www.thecurrent.org/playlist/2018-10-07/11 urls


/Users/tiverson/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/tiverson/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Starting http://www.thecurrent.org/playlist/2018-10-07/10 urls
Starting http://www.thecurrent.org/playlist/2018-10-07/09 urls
Starting http://www.thecurrent.org/playlist/2018-10-07/08 urls
Starting http://www.thecurrent.org/playlist/2018-10-07/07 urls
Starting http://www.thecurrent.org/playlist/2018-10-07/06 urls
Starting http://www.thecurrent.org/playlist/2018-10-07/05 urls
Starting http://www.thecurrent.org/playlist/2018-10-07/04 urls
Starting http://www.thecurrent.org/playlist/2018-10-07/03 urls
Starting http://www.thecurrent.org/playlist/2018-10-07/02 urls
Starting http://www.thecurrent.org/playlist/2018-10-07/01 urls
Starting http://www.thecurrent.org/playlist/2018-10-07/00 urls
Starting http://www.thecurrent.org/playlist/2018-10-06/23 urls
Starting http://www.thecurrent.org/playlist/2018-10-06/22 urls
Starting http://www.thecurrent.org/playlist/2018-10-06/21 urls
Starting http://www.thecurrent.org/playlist/2018-10-06/20 urls
Starting http://www.thecurrent.org/playlist/2018-10-06/

Starting http://www.thecurrent.org/playlist/2018-10-01/23 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/22 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/21 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/20 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/19 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/18 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/17 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/16 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/15 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/14 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/13 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/12 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/11 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/10 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/09 urls
Starting http://www.thecurrent.org/playlist/2018-10-01/

Processed 7285 songs
Processed 7290 songs
Processed 7295 songs
Processed 7300 songs
Processed 7305 songs
Processed 7310 songs
Processed 7315 songs
Processed 7320 songs
Processed 7325 songs
Processed 7330 songs
Processed 7335 songs
Processed 7340 songs
Processed 7345 songs
Processed 7350 songs
Processed 7355 songs
Processed 7360 songs
Processed 7365 songs
Processed 7370 songs
Processed 7375 songs
Processed 7380 songs
Processed 7385 songs
Processed 7390 songs
Processed 7395 songs
Processed 7400 songs
Processed 7405 songs
Processed 7410 songs
Processed 7415 songs
Processed 7420 songs
Processed 7425 songs
Processed 7430 songs
Processed 7435 songs
Processed 7440 songs
Processed 7445 songs
Processed 7450 songs
Processed 7455 songs
Processed 7460 songs
Processed 7465 songs
Processed 7470 songs
Processed 7475 songs
Processed 7480 songs
Processed 7485 songs
Processed 7490 songs
Processed 7495 songs
Processed 7500 songs
Processed 7505 songs
Processed 7510 songs
Processed 7515 songs
Processed 752

...

Processed 344660 songs
Processed 344665 songs
Processed 344670 songs
Processed 344675 songs
Processed 344680 songs
Processed 344685 songs
Processed 344690 songs
Processed 344695 songs
Processed 344700 songs
Processed 344705 songs
Processed 344710 songs
Processed 344715 songs
Processed 344720 songs
Processed 344725 songs
Processed 344730 songs
Processed 344735 songs
Processed 344740 songs
Processed 344745 songs
Processed 344750 songs
Processed 344755 songs
Processed 344760 songs
Processed 344765 songs
Processed 344770 songs
Processed 344775 songs
Processed 344780 songs
Processed 344785 songs
Processed 344790 songs
Processed 344795 songs
Processed 344800 songs
Processed 344805 songs
Processed 344810 songs
Processed 344815 songs
Processed 344820 songs
Processed 344825 songs
Processed 344830 songs
Processed 344835 songs
Processed 344840 songs
Processed 344845 songs
Processed 344850 songs
Processed 344855 songs
Processed 344860 songs
Processed 344865 songs
Processed 344870 songs
Processed 3

Processed 352155 songs
Processed 352160 songs
Processed 352165 songs
Processed 352170 songs
Processed 352175 songs
Processed 352180 songs
Processed 352185 songs
Processed 352190 songs
Processed 352195 songs
Processed 352200 songs
Processed 352205 songs
Processed 352210 songs
Processed 352215 songs
Processed 352220 songs
Processed 352225 songs
Processed 352230 songs
Processed 352235 songs
Processed 352240 songs
Processed 352245 songs
Processed 352250 songs
Processed 352255 songs
Processed 352260 songs
Processed 352265 songs
Processed 352270 songs
Processed 352275 songs
Processed 352280 songs
Processed 352285 songs
Processed 352290 songs
Processed 352295 songs
Processed 352300 songs
Processed 352305 songs
Processed 352310 songs
Processed 352315 songs
Processed 352320 songs
Processed 352325 songs
Processed 352330 songs
Processed 352335 songs
Processed 352340 songs
Processed 352345 songs
Processed 352350 songs
Processed 352355 songs
Processed 352360 songs
Processed 352365 songs
Processed 3

Processed 359655 songs
Processed 359660 songs
Processed 359665 songs
Processed 359670 songs
Processed 359675 songs
Processed 359680 songs
Processed 359685 songs
Processed 359690 songs
Processed 359695 songs
Processed 359700 songs
Processed 359705 songs
Processed 359710 songs
Processed 359715 songs
Processed 359720 songs
Processed 359725 songs
Processed 359730 songs
Processed 359735 songs
Processed 359740 songs
Processed 359745 songs
Processed 359750 songs
Processed 359755 songs
Processed 359760 songs
Processed 359765 songs
Processed 359770 songs
Processed 359775 songs
Processed 359780 songs
Processed 359785 songs
Processed 359790 songs
Processed 359795 songs
Processed 359800 songs
Processed 359805 songs
Processed 359810 songs
Processed 359815 songs
Processed 359820 songs
Processed 359825 songs
Processed 359830 songs
Processed 359835 songs
Processed 359840 songs
Processed 359845 songs
Processed 359850 songs
Processed 359855 songs
Processed 359860 songs
Processed 359865 songs
Processed 3

Processed 367150 songs
Processed 367155 songs
Processed 367160 songs
Processed 367165 songs
Processed 367170 songs
Processed 367175 songs
Processed 367180 songs
Processed 367185 songs
Processed 367190 songs
Processed 367195 songs
Processed 367200 songs
Processed 367205 songs
Processed 367210 songs
Processed 367215 songs
Processed 367220 songs
Processed 367225 songs
Processed 367230 songs
Processed 367235 songs
Processed 367240 songs
Processed 367245 songs
Processed 367250 songs
Processed 367255 songs
Processed 367260 songs
Processed 367265 songs
Processed 367270 songs
Processed 367275 songs
Processed 367280 songs
Processed 367285 songs
Processed 367290 songs
Processed 367295 songs
Processed 367300 songs
Processed 367305 songs
Processed 367310 songs
Processed 367315 songs
Processed 367320 songs
Processed 367325 songs
Processed 367330 songs
Processed 367335 songs
Processed 367340 songs
Processed 367345 songs
Processed 367350 songs
Processed 367355 songs
Processed 367360 songs
Processed 3

Processed 374650 songs
Processed 374655 songs
Processed 374660 songs
Processed 374665 songs
Processed 374670 songs
Processed 374675 songs
Processed 374680 songs
Processed 374685 songs
Processed 374690 songs
Processed 374695 songs
Processed 374700 songs
Processed 374705 songs
Processed 374710 songs
Processed 374715 songs
Processed 374720 songs
Processed 374725 songs
Processed 374730 songs
Processed 374735 songs
Processed 374740 songs
Processed 374745 songs
Processed 374750 songs
Processed 374755 songs
Processed 374760 songs
Processed 374765 songs
Processed 374770 songs
Processed 374775 songs
Processed 374780 songs
Processed 374785 songs
Processed 374790 songs
Processed 374795 songs
Processed 374800 songs
Processed 374805 songs
Processed 374810 songs
Processed 374815 songs
Processed 374820 songs
Processed 374825 songs
Processed 374830 songs
Processed 374835 songs
Processed 374840 songs
Processed 374845 songs
Processed 374850 songs
Processed 374855 songs
Processed 374860 songs
Processed 3